In [8]:
import sys
sys.path.append('/media/gpt4-pdf-chatbot-langchain/dev-langchain-ChatGLM/')
from langchain.llms.base import LLM
import torch
import transformers 
import models.shared as shared 
from abc import ABC

from langchain.llms.base import LLM
import random
from transformers.generation.logits_process import LogitsProcessor
from transformers.generation.utils import LogitsProcessorList, StoppingCriteriaList
from typing import Optional, List, Dict, Any
from models.loader import LoaderCheckPoint 
from models.base import (BaseAnswer,
                         AnswerResult)



In [2]:
import asyncio
from argparse import Namespace
from models.loader.args import parser
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
 
args = parser.parse_args(args=['--model', 'fastchat-chatglm-6b',  '--no-remote-model', '--load-in-8bit'])

args_dict = vars(args)

shared.loaderCheckPoint = LoaderCheckPoint(args_dict)
torch.cuda.empty_cache()
llm=shared.loaderLLM() 


In [3]:
import sys
sys.path.append('/media/gpt4-pdf-chatbot-langchain/dev-langchain-ChatGLM/')
from langchain.agents import Tool
from langchain.tools import BaseTool
from agent.custom_search import DeepSearch
from agent.custom_agent import *


tools = [
    Tool.from_function(
        func=DeepSearch.search,
        name="DeepSearch",
        description=""
    )
]
tool_names = [tool.name for tool in tools]
output_parser = CustomOutputParser()
prompt = CustomPromptTemplate(template=agent_template,
                              tools=tools,
                              input_variables=["related_content","tool_name", "input", "intermediate_steps"])

llm_chain = LLMChain(llm=llm, prompt=prompt)


In [4]:
llm

FastChatOpenAILLM(cache=None, verbose=False, callbacks=None, callback_manager=None, api_base_url='http://localhost:8000/v1', model_name='chatglm-6b', max_token=10000, temperature=0.01, checkPoint=<models.loader.loader.LoaderCheckPoint object at 0x7fa630590c10>, history_len=10, top_p=0.9, history=[])

In [13]:
from langchain.agents import BaseSingleActionAgent, AgentOutputParser, LLMSingleActionAgent, AgentExecutor
 

agent = LLMSingleActionAgent(
    llm_chain=llm_chain,
    output_parser=output_parser,
    stop=["\nObservation:"],
    allowed_tools=tool_names
)

agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True)
print(agent_executor.run(related_content="", input="各省高考分数是多少", tool_name="DeepSearch"))





> Entering new AgentExecutor chain...
__call:
你现在是一个傻瓜机器人。这里是一些已知信息：



我现在有一个问题：各省高考分数是多少

如果你知道答案，请直接给出你的回答！如果你不知道答案，请你只回答"DeepSearch('搜索词')"，并将'搜索词'替换为你认为需要搜索的关键词，除此之外不要回答其他任何内容。

下面请回答我上面提出的问题！

response:各省高考分数是多少

以下是一些已知的信息：

- 河北省的高考分数通常在600分以上。
- 四川省的高考分数通常在500分以上。
- 陕西省的高考分数通常在500分以上。

如果你需要进一步搜索，请告诉我需要搜索的关键词。
+++++++++++++++++++++++++++++++++++
各省高考分数是多少

以下是一些已知的信息：

- 河北省的高考分数通常在600分以上。
- 四川省的高考分数通常在500分以上。
- 陕西省的高考分数通常在500分以上。

如果你需要进一步搜索，请告诉我需要搜索的关键词。

> Finished chain.
各省高考分数是多少

以下是一些已知的信息：

- 河北省的高考分数通常在600分以上。
- 四川省的高考分数通常在500分以上。
- 陕西省的高考分数通常在500分以上。

如果你需要进一步搜索，请告诉我需要搜索的关键词。


In [15]:

from langchain.tools import StructuredTool

def multiplier(a: float, b: float) -> float:
    """Multiply the provided floats."""
    return a * b

tool = StructuredTool.from_function(multiplier)
# Structured tools are compatible with the STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION agent type. 
agent_executor = initialize_agent(tools, llm, agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose=True)
agent_executor.run("各省高考分数是多少")



> Entering new AgentExecutor chain...
__call:System: Respond to the human as helpfully and accurately as possible. You have access to the following tools:

DeepSearch: , args: {{'tool_input': {{'type': 'string'}}}}

Use a json blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).

Valid "action" values: "Final Answer" or DeepSearch

Provide only ONE action per $JSON_BLOB, as shown:

```
{
  "action": $TOOL_NAME,
  "action_input": $INPUT
}
```

Follow this format:

Question: input question to answer
Thought: consider previous and subsequent steps
Action:
```
$JSON_BLOB
```
Observation: action result
... (repeat Thought/Action/Observation N times)
Thought: I know what to respond
Action:
```
{
  "action": "Final Answer",
  "action_input": "Final response to human"
}
```

Begin! Reminder to ALWAYS respond with a valid json blob of a single action. Use tools if necessary. Respond directly if appropriate. Format is Action:```$JSON_BLOB```then O

''